### Camera caliberation code blocks
#### Ref
[OpenCV Camera Caliberation](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html#calibration)
#### Notes
1. Determine interinsic and extrinsic parameters of a camera to undistort images.
2. *Intrinsic parameters* - 
    - Parameters specific to camera including focal lenghts (fx,fy), camera center (Cx,Cy), also called as camera matrix.
    - Its a camera property, so once calculated can be stored and used for future usage.
    - 3x3 matrix ![Interinsic parameters](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/math/5e9d2db8b3561f68fa0688992fa45c13766a382c.png "Camera Matrix")
3. *Extrinsic parameters* - 
    - Combination of rotation and translation vectors to translate 3D world coordinates to camera coordinate system.
4. 
    - These distortions need to be corrected first by finding all these caliberation parameters. 
    - We generally do this by taking images having some **well-defined pattern** like checker-board, where we mark atleast 10 test points (*square corners etc.*) whose coordinates we know in *real-world space* as well as in the *2D image*.
    - We solve mathematical equations taking these 10 points and obtain distortion coefficints and interinsic/extrinsic parametrs.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#### Steps for finding caliberation matrices
1. For an image, detect chessBoardCorners by specifying what kind of pattern (8x8 grid OR 5x5 grid) we want to detect corners into, to `cv2.findChessboardCorners()` which returns corner points and retval (True if the pattern is obtained).
2. We should try the function on number of frames to get good pattern out of number of images.
3. `cv2.findCirclesGrid()` is used for detecting points out of circular pattern.
4. `cv2.cornerSubPix()` to increase accuracy of detected corner points.
5. `cv2.drawChessboardCorners()` to draw detected pattern on the image.
6. After having both the object-points (3D) and image-points(2D), we do caliberation using `cv2.caliberateCamera()` which retuens **Camera matrix, distortion coefficients and rotation-translation vectors**
7. Refine camera-matrix using `cv2.getOptimalNewCameraMatrix()`
8. Use `cv2.undistort()` to undistort image using refined camera-matrix and distortion-coefficients.
OR
9. find mapping b/w distorted and undistorted images using `cv2.initUndistortRectifyMap()` and apply the same using `cv2.remap()`
10. To estimate exactness of found parameters (interinsic, distortion, rotation and translation matrices), we reproject object-points to image points using these parameters and compare with actial image-points to get mean error.

### Epipolar Geometry
#### Ref
[OpenCV - Epipolar Geometry](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_epipolar_geometry/py_epipolar_geometry.html)
#### Notes
1. When image is taken from single-camera, we *lose depth information*, due to 3D-2D conversion. 
2. But we can *estimate depth information using more than one cameras*. E.g. eyes act as *stereo vision* (2 cameras). 
3. Epipolar geometry is associated with *multi-view geometry* and involves concepts like *epipole, epipolar lines, and epipolar constraints*.
4. With single camera, we can't determine exact 3-D points`(X)` corresponding to a point`(x)` in the image plane, as *every point on the line joining camera-center`(O)` and object-point will project to the same point on the image plane*. But with a *second camera capturing same scene, that line`(OX)` projects to a line`(l')` on 2nd camera's image plane*. So with these 2 images, *we can determine the correct 3D/object-point by traingulation*.
5. **Epiline** 
    - Projection of line `OX` to 2nd camera's image plane `l'`. `l'` is the *epline* corresponding to the point `x` of 1st image. 
    - **Significance** To search corresponding point of `x` on the 2nd image, we need not search whole image, instead just the corresponding epiline. (also called *epipolar constraint*)
    - plane corresponding to 2 camera-centers and object-point is called **Epipolar plane**
6. **Epipole**
    - Projection of a camera-center on the image plane of another. Epipole is the point of intersection of line through camera center and image planes. 
    - Sometimes camera-center doesn't project onto another image plane, ie. one camera doesn't see the other.
    - *Epiline always passes through its epipole* i.e. epipole is the interention point of epilines. So we can find many epilines and their intersection point to get the epipole.
    - To determine epipoles and epilines, we need to find **Fundamental Matrix (F)** and **Essential matrix (E)**.
7. **Essential matrix**
    - Contains information on translation and rotation for determining location of 2nd camera w.r.t. 1st camera in the global coordinates.
8. **Fundamental matrix**
    - Contains same information as the *Essential Matrix* and also interinsics of both the cameras to map a point of an image to corresponding epiline in the 2nd image.
    - We need 8 corresponding/matching points to determine the fundamental matrix, using 8-point algorithm, although more number of matching points are preferred. RANSAC technique can be used to get robust set of matching points

$e^{i\pi} + 1 = 0$